## Confusion Matrix

In [1]:
import re
import matplotlib.pyplot as plt
import itertools
import pandas as pd
from copy import deepcopy
from sklearn.metrics import confusion_matrix
import numpy as np

/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/cnsetzer/miniconda3/envs/astrotog/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


* Object
* RA, DEC
* delta t lambda (for detected objects)
* Milky extinction
* Redshift
* nObs per pass-band
* Peak-mag per pass-band

In [2]:
# dataframe( columns = ['true', 'predicted', 'RA', 'Dec', 'prop'])

#def transientMetricPlots(y_true, y_pred, RA, DEC, prop):
def transientMetricPlots(dataframe,property_name):
    nbins = 150
    prop_list = []
    logloss_list = []
    # Compute binning
        # values in bins
    dataframe['{}'.format(property_name)].dropna(inplace=True)
    hist, bin_edges = np.histogram(dataframe['{}'.format(property_name)],bins=nbins) 

    for i in range(nbins):
        bin_value = (bin_edges[i+1] + bin_edges[i])/2.0
        dataframe.dropna(inplace=True, subset=['{}'.format(property_name)])
        if re.search('nobs',property_name) is not None:
            bin_value=np.ceil(bin_value)
            property_subset = dataframe.query('{0} <= {2}  & {2} < {1}'.format(bin_edges[i],bin_edges[nbins-1],property_name))
        else:
            property_subset = dataframe.query('{0} <= {2}  & {2} < {1}'.format(bin_edges[i],bin_edges[i+1],property_name))
        if property_subset.empty:
            continue
        else:
            y_true = np.vstack(property_subset['true']) #y_true.values
            y_pred = np.vstack(property_subset['predicted'])

            y_preds = np.argmax(y_pred, axis=1)
            y_trues = np.argmax(y_true, axis=1)

            cm = confusion_matrix(y_trues, y_preds)
            #print("Confusion Matrix:\n{}".format(cm))

            logloss = log_loss(y_true, y_pred)
            logloss_list.append(logloss)
            prop_list.append(bin_value)
            #print("Log-Loss:\n{}".format(logloss))
    
    # plot 2D classification metric (log-loss) vs property
    fig2d, axes = plt.subplots(2,1,sharex=True,gridspec_kw={'height_ratios':[5,2]})
    axes[0].scatter(prop_list,logloss_list)
    axes[0].set_title('Classification Log-loss vs. {}'.format(property_name))
    axes[0].set_xlabel('{}'.format(property_name))
    axes[0].set_ylabel('Log-loss')
    
    axes[1].hist(dataframe['{}'.format(property_name)], bins=nbins)
    axes[1].set_yscale('log')
    axes[1].set_xlabel('{}'.format(property_name))
    axes[1].set_ylabel('N({})'.format(property_name))
    #To Do: - FORMAT PLOTS
    return fig2d
  
    # stretch goal: heat map on sky for fun

In [3]:
from sklearn.metrics import log_loss

# inputs: confusion matrix
# inputs: properties, RA, DEC

pred_path = "pred_DDF.csv"
true_path = "truth_DDF.csv"
prop_path = 'obj_table_DDF.txt'
    
y_true = pd.read_csv(true_path, delimiter='\s+', dtype='f', index_col= False, header=None)
y_pred = pd.read_csv(pred_path, delimiter='\s+', dtype='f', index_col=False, header=None)
obj_table = pd.read_csv(prop_path, sep='\s+',index_col=0)

In [4]:
true_pred_df = pd.DataFrame(index=obj_table.index,columns=['true','predicted'])

for k, ind in enumerate(list(obj_table.index)):
    true_pred_df.at[ind,'true'] = np.asarray(y_true.iloc[k])
    true_pred_df.at[ind,'predicted'] = np.asarray(y_pred.iloc[k])

In [5]:
dataframe = obj_table.join(true_pred_df)

In [6]:
print(dataframe.columns)

Index(['peakmjd', 'trigger_mjd', 'sim_host_redshift', 'classnum', 'ra', 'dec',
       'mwebv', 'pu', 'pg', 'pr', 'pi', 'pz', 'pY', 'nobs_u', 'nobs_g',
       'nobs_r', 'nobs_i', 'nobs_z', 'nobs_Y', 'deltat_u', 'deltat_g',
       'deltat_r', 'deltat_i', 'deltat_z', 'deltat_Y', 'true', 'predicted'],
      dtype='object')


In [7]:
for property_name in list(dataframe.columns):
    if re.search('true',property_name) is None and re.search('predicted',property_name) is None and re.search('ra',property_name) is None and re.search('dec',property_name) is None and re.search('classnum',property_name) is None: 
        print(property_name)
        figure2d = transientMetricPlots(deepcopy(dataframe),property_name)
        figure2d.savefig('ddf_classification_vs_{}.pdf'.format(property_name), bbox_inches='tight')
        plt.close(figure2d)

peakmjd
trigger_mjd
sim_host_redshift
mwebv
pu
pg
pr
pi
pz
pY
nobs_u
nobs_g
nobs_r
nobs_i
nobs_z
nobs_Y
deltat_u
deltat_g
deltat_r
deltat_i
deltat_z
deltat_Y
